In [1]:
import requests
import os
import time
import sys
import json
import math
from collections import defaultdict

# --- 1. CONFIGURACIÓN DEL SCRIPT ---
# ==========================================================
# ¡¡DEFINE EL TIEMPO TOTAL QUE QUIERES QUE CORRA EL SCRIPT (en minutos)!!
#
DURACION_TOTAL_MINUTOS = 300# Por ejemplo, 120 minutos (2 horas)
#
# ==========================================================

# Convertir la duración a segundos para la lógica interna
DURACION_TOTAL_SEGUNDOS = DURACION_TOTAL_MINUTOS * 60

# --- Rutas de Archivos ---
INPUT_FILE_PATH = "/kaggle/input/mi-lista-de-descarga/descarga_lista.jsonl"
BASE_OUTPUT_FOLDER = "/kaggle/working/imagenes_muestreadas_por_tiempo"
METADATA_OUTPUT_FILE = "/kaggle/working/metadata_muestra_por_tiempo.json"

# --- 2. LEER Y PRE-PROCESAR EL ARCHIVO JSONL ---
print(f"Iniciando Muestreo: El script se ejecutará por {DURACION_TOTAL_MINUTOS} minutos.")
print(f"Leyendo archivo: {INPUT_FILE_PATH}")

try:
    with open(INPUT_FILE_PATH, 'r') as f:
        all_images_data = [json.loads(line) for line in f if line.strip()]
except FileNotFoundError:
    print(f"Error: No se encontró '{INPUT_FILE_PATH}'.")
    sys.exit()

total_disponible = len(all_images_data)
if not all_images_data:
    print("El archivo JSONL está vacío. Saliendo.")
    sys.exit()

print(f"Se leerán {total_disponible} imágenes candidatas.")

# --- Organizar todas las imágenes por BBOX ---
data_by_bbox = defaultdict(list)
for img_data in all_images_data:
    try:
        data_by_bbox[img_data['bbox_index']].append(img_data)
    except KeyError:
        pass # Ignorar líneas corruptas

total_bboxes_con_imagenes = len(data_by_bbox)
print(f"Imágenes organizadas en {total_bboxes_con_imagenes} BBOXes únicos.")


# --- 3. BUCLE COMBINADO DE SELECCIÓN Y DESCARGA POR TIEMPO ---
print(f"\nIniciando descarga. Tiempo límite: {DURACION_TOTAL_MINUTOS} minutos.")
os.makedirs(BASE_OUTPUT_FOLDER, exist_ok=True)

# Registrar el tiempo de inicio y calcular el tiempo de finalización
script_start_time = time.time()
script_end_time = script_start_time + DURACION_TOTAL_SEGUNDOS

metadata_procesada = [] # Aquí guardaremos la metadata de las imágenes que procesamos
current_depth = 0       # Profundidad/Ronda del "Round Robin"
processed_count = 0     # Contador de imágenes procesadas

try:
    # El bucle principal ahora se basa en el TIEMPO
    while time.time() < script_end_time:
        
        images_added_in_this_round = 0
        print(f"\nIniciando Ronda 'Round Robin' Nivel {current_depth}...")
        
        # Damos una vuelta por todos los BBOXes (ordenados del 1 al 120...)
        for bbox_index in sorted(data_by_bbox.keys()):
            
            # --- VERIFICACIÓN DE TIEMPO (INTERNA) ---
            # Verificamos el tiempo ANTES de procesar cada BBOX
            if time.time() >= script_end_time:
                print("\n¡Tiempo límite alcanzado! Deteniendo el bucle 'for' interno.")
                break # Romper el bucle 'for' interno
            
            # ¿Este BBOX todavía tiene una imagen en esta "ronda" (profundidad)?
            if current_depth < len(data_by_bbox[bbox_index]):
                
                # ¡Sí! La tomamos para procesar.
                img_data = data_by_bbox[bbox_index][current_depth]
                metadata_procesada.append(img_data) # La añadimos a la metadata
                images_added_in_this_round += 1
                processed_count += 1
                
                # Imprimir barra de progreso
                if processed_count % 5 == 0:
                    tiempo_restante = script_end_time - time.time()
                    print(f"Progreso: {processed_count} imágenes procesadas... Tiempo restante: {tiempo_restante:.0f}s   ", end="\r")
                
                # --- FASE DE DESCARGA INMEDIATA ---
                try:
                    download_url = img_data['download_url']
                    filename = img_data['filename']
                    file_path = os.path.join(BASE_OUTPUT_FOLDER, filename)
                    
                    if os.path.exists(file_path):
                        continue
                        
                    img_response = requests.get(download_url, timeout=10)
                    if img_response.status_code == 200:
                        with open(file_path, 'wb') as img_f:
                            img_f.write(img_response.content)
                            
                except Exception as e:
                    print(f"\nError al descargar ID {img_data.get('id', 'N/A')}. Omitiendo: {e}")
                    pass
                # --- Fin de la descarga ---

        # --- VERIFICACIÓN DE TIEMPO (EXTERNA) ---
        # Verificamos el tiempo después de completar una ronda entera
        if time.time() >= script_end_time:
            print("\n¡Tiempo límite alcanzado! Deteniendo el bucle 'while' externo.")
            break # Romper el bucle 'while' externo
        
        # ¿Nos quedamos sin imágenes antes de tiempo?
        if images_added_in_this_round == 0:
            print(f"\nSe agotaron todas las imágenes disponibles ({processed_count} procesadas) antes del límite de tiempo.")
            break # No hay más imágenes que añadir

        # Pasamos a la siguiente ronda (ej. buscar la 2da imagen de cada BBOX)
        current_depth += 1

except KeyboardInterrupt:
    print("\n¡Interrupción manual detectada! Pasando al reporte final.")
    pass # Permite al usuario detener con Ctrl+C y aun así guardar el reporte

# --- 4. REPORTE FINAL Y GUARDADO DE METADATA ---
script_end_runtime = time.time()
total_runtime_segundos = script_end_runtime - script_start_time
total_descargado = len(os.listdir(BASE_OUTPUT_FOLDER)) # Contar archivos reales

print(f"\nProceso detenido. Guardando metadata de {len(metadata_procesada)} imágenes procesadas...")

# Guardar el archivo de metadata en la carpeta de output
print(f"Guardando metadata en '{METADATA_OUTPUT_FILE}'...")
with open(METADATA_OUTPUT_FILE, 'w') as meta_f:
    json.dump(metadata_procesada, meta_f, indent=2)

print("\n" + "="*30)
print("--- MUESTREO POR TIEMPO COMPLETADO ---")
print(f"Tiempo Objetivo: {DURACION_TOTAL_MINUTOS:.2f} minutos.")
print(f"Tiempo Real de Ejecución: {total_runtime_segundos / 60:.2f} minutos ({total_runtime_segundos:.0f} segundos).")
print(f"Imágenes Únicas Descargadas: {total_descargado}")
print(f"Total Imágenes Procesadas (Metadata): {len(metadata_procesada)}")
print(f"Imágenes guardadas en: '{BASE_OUTPUT_FOLDER}'")
print(f"Metadata guardada en: '{METADATA_OUTPUT_FILE}'")
print("="*30)

Iniciando Muestreo: El script se ejecutará por 300 minutos.
Leyendo archivo: /kaggle/input/mi-lista-de-descarga/descarga_lista.jsonl
Se leerán 295404 imágenes candidatas.
Imágenes organizadas en 93 BBOXes únicos.

Iniciando descarga. Tiempo límite: 300 minutos.

Iniciando Ronda 'Round Robin' Nivel 0...
Progreso: 90 imágenes procesadas... Tiempo restante: 17946s   
Iniciando Ronda 'Round Robin' Nivel 1...
Progreso: 185 imágenes procesadas... Tiempo restante: 17889s   
Iniciando Ronda 'Round Robin' Nivel 2...
Progreso: 275 imágenes procesadas... Tiempo restante: 17835s   
Iniciando Ronda 'Round Robin' Nivel 3...
Progreso: 370 imágenes procesadas... Tiempo restante: 17729s   
Iniciando Ronda 'Round Robin' Nivel 4...
Progreso: 465 imágenes procesadas... Tiempo restante: 17613s   
Iniciando Ronda 'Round Robin' Nivel 5...
Progreso: 555 imágenes procesadas... Tiempo restante: 17503s   
Iniciando Ronda 'Round Robin' Nivel 6...
Progreso: 650 imágenes procesadas... Tiempo restante: 17388s   
Ini